In [ ]:
import sys
import glob
import pandas as pd

In [ ]:
csvdoor=glob.glob("*door*.pcap.csv")
csvdoor.sort()
csvdoor

In [ ]:
csvidle=glob.glob("*idle*.pcap.csv")
csvidle.sort()
csvidle

In [ ]:
pd.read_csv(csvdoor[0], sep="|")['frame.len'].max()

In [ ]:
csvinfo = {}
for csv in csvdoor:
    csvinfo[csv] = {}
    csvinfo[csv]['df'] = pd.read_csv(csv, sep="|")
    csvinfo[csv]['door'] = True
    csvinfo[csv]['pktcount'] = csvinfo[csv]['df'].shape[0]
for csv in csvidle:
    csvinfo[csv] = {}
    csvinfo[csv]['df'] = pd.read_csv(csv, sep="|")
    csvinfo[csv]['door'] = False
    csvinfo[csv]['pktcount'] = csvinfo[csv]['df'].shape[0]
csvinfo['fridge_door2.pcap.csv']['pktcount']

In [ ]:
csvmeta = pd.DataFrame(columns=['csvfile', 'df', 'door', 'pkt_count', 'size_door', 'size_idle', 'size', 'maxs'])
for csv in csvdoor:
    csvdf = pd.read_csv(csv, sep="|")
    size  = csvdf['frame.len'].sum()
    maxs  = csvdf['frame.len'].max()
    csvmeta = csvmeta.append({
        'csvfile': csv, 
        'df': csvdf,
        'door': True,
        'pkt_count': csvdf.shape[0],
        'size_door': size,
        'size': size,
        'maxs': maxs
    }, ignore_index=True)
for csv in csvidle:
    csvdf=pd.read_csv(csv, sep="|")
    size  = csvdf['frame.len'].sum()
    maxs  = csvdf['frame.len'].max()
    csvmeta = csvmeta.append({
        'csvfile': csv, 
        'df': csvdf,
        'door': False,
        'pkt_count': csvdf.shape[0],
        'size_idle': size,
        'size': size,
        'maxs': maxs
    }, ignore_index=True)
csvmeta

In [ ]:
ax = csvmeta.plot.barh(x='csvfile', y=['size_idle', 'size_door'], stacked=True)

In [ ]:
from bokeh.io import output_file, show, save
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, HoverTool
import json
from IPython.display import IFrame
import math

In [ ]:
def merc(Coords, origin):

    if Coords is None:
        return (0,0) 

    lat = Coords['lat']
    lng = Coords['lng']
  
    if (lat == 0.0 and lng == 0.0):
        if origin:
            lat = origin['lat']
            lng = origin['lng']
        else:
            return (0, 0)

    r_major = 6378137.000
    x = r_major * math.radians(lng)
    scale = x/lng
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
# Where are you now? (this is optional, just to plot some cool lines)
origin = {'lat':41.795,'lng':-87.60}

In [ ]:
def map_plot(coords, origin = None, title = None):
    """Plot multiple points in a world map using array of dict coordinates.

    Keyword arguments:
    coords -- array of dict objects containing lat,lng,color 
              eg. [{lat: 41.795, lng: -87.60, color: 'green'}]
    origin -- coordinates for your origin location (default None)
    """
    
    if origin:
        o = merc(origin, origin)
        x_origin = [o[0]]
        y_origin = [o[1]]
        
    tile_provider = get_provider(Vendors.CARTODBPOSITRON)

    ### Hover Tool ###
    source = ColumnDataSource(data=coords)
    
    # TODO: enable the hover tool to display ISP information
    hover = HoverTool(tooltips=[
        ("isp", "@isp"),
        ("oui","@oui")
    ])

    p = figure(x_range=(-18780000, 18000000), y_range=(-1000000, 7000000),
               x_axis_type="mercator", y_axis_type="mercator", 
               plot_width = 900, plot_height = 500, title = title,
               tools=[hover, '','box_zoom','wheel_zoom','save'],
               toolbar_location="below") 
    #https://docs.bokeh.org/en/latest/docs/user_guide/tools.html
    
    p.add_tile(tile_provider)

    if origin:
        p.circle(x=x_origin, y=y_origin, size=10, color="black")

    ###
    # plot multiple lines coming from the origin to dest. coordinates
    #
    if origin:
        p.multi_line(xs='xs',
                     ys='ys',
                     color='color',
                     line_width=2, source=source)
    ###
    # only plot point coordinates
    #

    p.circle(x='x', y='y', size=10, color='color', source=source)

    output_file("tile1.html")
    save(p)
    return IFrame(src='./tile1.html', width=950, height=520)
    

#map_plot (coords, origin, title='TCP/UDP End Points')

In [ ]:
for k in csvdict.keys():
    print(csvdict[k]['df'].head(1))